# LSTM tutorial

In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd

In [3]:
torch.manual_seed(1)  # 랜덤 시드 고정

In [5]:
lstm = nn.LSTM(3, 3) # 입력차원 3, 출력차원 3
inputs = [autograd.Variable(torch.randn(1, 3))
          for _
          in range(5)
         ]   # 1*3 의 입력 벡터 5개

In [8]:
# 히든 스테이트 초기화
hidden = (autograd.Variable(torch.randn(1, 1, 3)),
          autograd.Variable(torch.randn(1, 1, 3)))

In [11]:
for i in inputs:
    # 한 time에 한 엘리먼트를 시퀀스에 넘김
    # 각 스텝 이후에 히든 변수는 히든 스테이트를 포함한다.
    out, hidden = lstm(i.view(1, 1, -1), hidden)
    
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (autograd.Variable(torch.randn(1, 1, 3)), autograd.Variable(
    torch.randn(1, 1, 3))) 

out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)

Variable containing:
(0 ,.,.) = 
 -0.0638 -0.1500 -0.0040

(1 ,.,.) = 
 -0.1819 -0.0203  0.1032

(2 ,.,.) = 
 -0.3154 -0.0581  0.0973

(3 ,.,.) = 
 -0.2813 -0.1581  0.0831

(4 ,.,.) = 
 -0.2590  0.0798  0.1105
[torch.FloatTensor of size 5x1x3]

(Variable containing:
(0 ,.,.) = 
 -0.2590  0.0798  0.1105
[torch.FloatTensor of size 1x1x3]
, Variable containing:
(0 ,.,.) = 
 -0.6811  0.1010  0.2929
[torch.FloatTensor of size 1x1x3]
)


In [13]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    tensor = torch.LongTensor(idxs)
    return autograd.Variable(tensor)

training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]

word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

EMBEDDING_DIM = 6
HIDDEN_DIM = 6


{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [21]:
# 모델을 만들자

class LSTMTagger(nn.Module):
    
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()
    
    def init_hidden(self):
        return (autograd.Variable(torch.zeros(1, 1, self.hidden_dim)),
                autograd.Variable(torch.zeros(1, 1, self.hidden_dim))
               )
    
    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(
            embeds.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_score = F.log_softmax(tag_space, dim=1)
        return tag_score

In [22]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [23]:
inputs = prepare_sequence(training_data[0][0], word_to_ix)
tag_scores = model(inputs)
print(tag_scores)

Variable containing:
-0.9885 -1.0338 -1.3012
-1.0772 -0.8958 -1.3817
-1.1170 -0.8647 -1.3801
-1.0864 -0.8939 -1.3723
-1.0390 -0.9786 -1.3080
[torch.FloatTensor of size 5x3]



In [24]:
for epoch in range(300):
    for sentence, tags in training_data:
        model.zero_grad()
        model.hidden = model.init_hidden()
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)
        
        # forward pass
        tag_scores = model(sentence_in)
        
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

In [25]:
inputs = prepare_sequence(training_data[0][0], word_to_ix)
tag_scores = model(inputs)

In [26]:
print(tag_scores)

Variable containing:
-0.0933 -2.4564 -5.7051
-4.5787 -0.0250 -4.2363
-4.1484 -3.5775 -0.0447
-0.0655 -2.9250 -4.6341
-2.5836 -0.1208 -3.2624
[torch.FloatTensor of size 5x3]

